In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
import os
import sys 

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [3]:
!echo $PYSPARK_PYTHON

/Users/oluwadaraadedeji/anaconda3/bin/python


In [4]:
!echo $PYSPARK_DRIVER_PYTHON

/Users/oluwadaraadedeji/anaconda3/bin/python


In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/02/27 19:25:46 WARN Utils: Your hostname, Oluwadaras-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.18.29.242 instead (on interface en0)
24/02/27 19:25:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/27 19:25:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/fhvhv/fhvhv_tripdata_2021-01.csv

--2024-02-27 18:27:35--  https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/fhvhv/fhvhv_tripdata_2021-01.csv
Resolving github.com (github.com)... 4.208.26.197
Connecting to github.com (github.com)|4.208.26.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021     [ <=>                ] 150.47K  --.-KB/s    in 0.02s   

2024-02-27 18:27:35 (7.46 MB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [154082]



In [6]:
!wc -l fhvhv_tripdata_2021-01.csv

 11908469 fhvhv_tripdata_2021-01.csv


In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [8]:
df.schema

24/02/27 19:26:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [9]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
!wc -l head.csv

     101 head.csv


In [11]:
import pandas as pd

In [12]:
df_pandas = pd.read_csv("head.csv")

In [13]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [14]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

StructType(
    StructField('hvfhs_license_num', StringType(), True), 
    StructField('dispatching_base_num', StringType(), True), 
    StructField('pickup_datetime', StringType(), True), 
    StructField('dropoff_datetime', StringType(), True), 
    StructField('PULocationID', IntegerType(), True), 
    StructField('DOLocationID', IntegerType(), True), 
    StructField('SR_Flag', StringType(), True)
    )

In [15]:
from pyspark.sql import types

In [16]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [17]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [18]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [19]:
df = df.repartition(24)

In [21]:
df.write.parquet('fhvhv/2021/01', mode ="overwrite")

24/02/27 19:27:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/27 19:27:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/27 19:27:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/27 19:27:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [22]:
df = spark.read.parquet('fhvhv/2021/01/')

In [23]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [24]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [25]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID','DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-04 09:54:16|2021-01-04 10:20:02|         152|         193|
|2021-01-03 16:45:51|2021-01-03 16:58:38|         208|         213|
|2021-01-04 15:51:39|2021-01-04 16:03:16|         145|         202|
|2021-01-02 08:15:37|2021-01-02 08:19:39|          78|         169|
|2021-01-01 17:46:05|2021-01-01 18:10:31|         181|         107|
|2021-01-02 19:36:41|2021-01-02 19:43:00|         188|          72|
|2021-01-01 16:05:01|2021-01-01 16:26:27|          61|           4|
|2021-01-03 00:05:40|2021-01-03 00:13:45|         257|          54|
|2021-01-04 16:37:58|2021-01-04 16:46:13|          75|          75|
|2021-01-01 23:42:33|2021-01-02 00:06:29|         147|         175|
|2021-01-05 08:31:45|2021-01-05 08:38:00|          81|          81|
|2021-01-01 18:57:24|2021-01-01 19:01:41|       

In [26]:
from pyspark.sql import functions as F

In [27]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [28]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [29]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()


+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b3b| 2021-01-04|  2021-01-04|         152|         193|
|  e/b14| 2021-01-03|  2021-01-03|         208|         213|
|  e/b30| 2021-01-04|  2021-01-04|         145|         202|
|  e/95b| 2021-01-02|  2021-01-02|          78|         169|
|  e/9ce| 2021-01-02|  2021-01-02|          17|          73|
|  e/a39| 2021-01-01|  2021-01-01|         181|         107|
|  e/b38| 2021-01-02|  2021-01-02|         188|          72|
|  s/b13| 2021-01-01|  2021-01-01|          61|           4|
|  a/b49| 2021-01-03|  2021-01-03|         257|          54|
|  e/b47| 2021-01-04|  2021-01-04|          75|          75|
|  e/b35| 2021-01-01|  2021-01-02|         147|         175|
|  e/b32| 2021-01-05|  2021-01-05|          81|          81|
|  e/b42| 2021-01-01|  2021-01-01|          60|          60|
|  e/b48| 2021-01-04|  2

In [31]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [32]:
!head -n 10 head.csv